In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
%matplotlib ipympl

import sys
import numpy as np
import time
from tqdm import tqdm
import ot

sys.path.append('./src')
verbose = False

random_seed = np.random.randint(100)
random_seed

19

In [2]:
from src.utils_solver import Lmatrix2paths, adapted_empirical_measure, adapted_wasserstein_squared, qpath2mu_x, quantization, nested, plot_V, sort_qpath
n_sample = 100
T = 3
L = np.array([[1, 0, 0], [2, 4, 0], [3, 2, 1]])
X,A = Lmatrix2paths(L, n_sample, seed = random_seed, verbose = False)
M = np.array([[1, 0, 0], [2, 3, 0], [3, 1, 2]])
Y,B = Lmatrix2paths(M, n_sample, seed = random_seed, verbose = False)

adaptedX = adapted_empirical_measure(X, delta_n = 0.1)
adaptedY = adapted_empirical_measure(Y, delta_n = 0.1)

# Quantization map
q2v = np.unique(np.concatenate([adaptedX, adaptedY], axis=0))
v2q = {k: v for v, k in enumerate(q2v)}  # Value to Quantization

# Quantized paths
qX = np.array([[v2q[x] for x in y] for y in adaptedX])
qY = np.array([[v2q[x] for x in y] for y in adaptedY])

# Sort paths and transpose to (n_sample, T+1)
qX = sort_qpath(qX.T)
qY = sort_qpath(qY.T)

mu_x = qpath2mu_x(qX)
nu_y = qpath2mu_x(qY)


In [3]:
# All list except weights should be increasing! 

mu_x_c = [list(mu_x_t.keys()) for mu_x_t in mu_x] # 
mu_x_n= [[len(d) for d in mu_x_t.values()] for mu_x_t in mu_x]
mu_x_cumn = [np.cumsum([0] + mu_x_n_t) for mu_x_n_t in mu_x_n]

mu_x_v= [[list(d.keys()) for d in mu_x_t.values()] for mu_x_t in mu_x]
mu_x_w = [[list(d.values()) for d in mu_x_t.values()] for mu_x_t in mu_x]

nu_y_c = [list(nu_y_t.keys()) for nu_y_t in nu_y]
nu_y_n= [[len(d) for d in nu_y_t.values()] for nu_y_t in nu_y]
nu_y_cumn = [np.cumsum([0] + nu_y_n_t) for nu_y_n_t in nu_y_n]

nu_y_v = [[list(d.keys()) for d in nu_y_t.values()] for nu_y_t in nu_y]
nu_y_w = [[list(d.values()) for d in nu_y_t.values()] for nu_y_t in nu_y]

In [4]:
square_cost_matrix = (q2v[None, :] - q2v[None, :].T) ** 2
V = [np.zeros([len(mu_x_c[t]),len(nu_y_c[t])]) for t in range(T)] # V_t(x_{1:t},y_{1:t})
t = 1
i, j = 10, 20
cx = mu_x_c[t][i]
cy = nu_y_c[t][j]
vx = mu_x_v[t][i]
vy = nu_y_v[t][j]

cost = square_cost_matrix[np.ix_(vx, vy)]

print(cx, cy)

cost += V[t+1][mu_x_cumn[t][i]:mu_x_cumn[t][i+1],nu_y_cumn[t][j]:nu_y_cumn[t][j+1]]

mu_x_c[t+1][mu_x_cumn[t][i]:mu_x_cumn[t][i+1]], nu_y_c[t+1][nu_y_cumn[t][j]:nu_y_cumn[t][j+1]]

(78, 72) (78, 82)


([(78, 72, 26), (78, 72, 56), (78, 72, 96)],
 [(78, 82, 101), (78, 82, 103), (78, 82, 106), (78, 82, 114)])